In [161]:
import numpy as np
import pandas as pd

In [162]:
# load anime dataset
anime_df = pd.read_csv('dataset/anime.csv')
usecols = ["MAL_ID", "Name", "Score", "Genres", "Type", "Episodes", "Premiered",
           "Studios", "Source", "Rating", "Members", "Completed"]

anime_df=pd.read_csv('dataset/anime.csv',usecols=usecols)
anime_df.head()

,MAL_ID,Name,Score,Genres,Type,Episodes,Premiered,Studios,Source,Rating,Members,Completed
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",TV,26,Spring 1998,Sunrise,Original,R - 17+ (violence & profanity),1251960,718161
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Movie,1,Unknown,Bones,Original,R - 17+ (violence & profanity),273145,208333
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",TV,26,Spring 1998,Madhouse,Manga,PG-13 - Teens 13 or older,558913,343492
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, Magic",TV,26,Summer 2002,Sunrise,Original,PG-13 - Teens 13 or older,94683,46165
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",TV,52,Fall 2004,Toei Animation,Manga,PG - Children,13224,7314


In [163]:
# load top users with anime rating dataset
rating_popular_anime = pd.read_csv('dataset/top_anime_unsupervised_use.csv')
rating_data = rating_popular_anime.merge(anime_df, left_on = 'MAL_ID', right_on = 'MAL_ID', how = 'left')
rating_data = rating_data[["user_id", "Name", "MAL_ID","rating"]]
combine_anime_rating = rating_data.dropna(axis = 0, subset = ['Name'])
anime_ratingCount = (combine_anime_rating.groupby(by = ['Name'])['rating'].count().reset_index()[['Name', 'rating']])
rating_data = combine_anime_rating.merge(anime_ratingCount, left_on = 'Name', right_on = 'Name', how = 'left')
rating_data = rating_data.drop(columns = "rating_x")
rating_data = rating_data.rename(columns={"rating_y": "rating"})
rating_data.head()

,user_id,Name,MAL_ID,rating
0,478,Witch Hunter Robin,7,853
1,478,Eyeshield 21,15,864
2,478,Hachimitsu to Clover,16,1374
3,478,Naruto,20,2211
4,478,Tennis no Ouji-sama,22,821


In [198]:
top_animes_count = 1000
top_users_count = 3000
count_group = rating_data.groupby('user_id')['rating'].count()
top_users_process = count_group.dropna().sort_values(ascending=False)[:top_users_count]
top_r = rating_data.join(top_users_process, rsuffix='_r', how='inner', on='user_id')

count_group = rating_data.groupby('MAL_ID')['rating'].count()
top_animes = count_group.dropna().sort_values(ascending=False)[:top_animes_count]
top_r = top_r.join(top_animes, rsuffix='_r', how='inner', on='MAL_ID')

pivot = pd.crosstab(top_r.user_id, top_r.MAL_ID, top_r.rating, aggfunc=np.sum)
pivot.fillna(0, inplace=True)
piviot_table = rating_data.pivot_table(index="Name",columns="user_id", values="rating").fillna(0)
piviot_table.head()

user_id,478,781,853,890,912,943,1177,1231,1393,1397,...,352366,352478,352660,352669,352761,352832,352835,352922,352924,352930
Name,,,,,,,,,,,,,,,,,,,,,
"""0""",0.0,0.0,0.0,307.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,307.0,0.0,0.0
"""Aesop"" no Ohanashi yori: Ushi to Kaeru, Yokubatta Inu",0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""Bungaku Shoujo"" Kyou no Oyatsu: Hatsukoi",0.0,0.0,0.0,0.0,0.0,0.0,0.0,922.0,0.0,922.0,...,0.0,922.0,0.0,0.0,922.0,0.0,0.0,0.0,0.0,922.0
"""Bungaku Shoujo"" Memoire",0.0,0.0,0.0,0.0,0.0,0.0,0.0,982.0,0.0,982.0,...,0.0,0.0,982.0,982.0,982.0,0.0,0.0,982.0,0.0,982.0
"""Bungaku Shoujo"" Movie",0.0,0.0,0.0,0.0,0.0,0.0,0.0,1196.0,0.0,1196.0,...,0.0,1196.0,1196.0,1196.0,1196.0,0.0,0.0,0.0,1196.0,1196.0


In [191]:
from scipy.sparse import csr_matrix
piviot_table_matrix = csr_matrix(piviot_table.values)

In [192]:
# create knn model and fit matrix
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(metric="cosine", algorithm="brute")
model.fit(piviot_table_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [201]:
def predict(anime_name_input, n_neighbors):
    query = piviot_table.loc[anime_name_input, :].values.reshape(1, -1)

    distance, suggestions = model.kneighbors(query, n_neighbors)
    recommendation = {}
    for i in range(0, len(distance.flatten())):
        if i != 0:
            recommendation[piviot_table.index[suggestions.flatten()[i]]] = distance.flatten()[i]

    return recommendation

In [202]:
predict('To Heart: Remember My Memories', 10)

{'To Heart': 0.2732727223710337,
 'To Heart: Remember My Memories Specials': 0.29498835462217965,
 'To Heart 2': 0.30237780857340457,
 'To Heart 2 OVA': 0.35235157990445853,
 'To Heart 2 AD': 0.3677287767366777,
 'To Heart 2 AD Plus': 0.38812794166617737,
 'To Heart 2 Special': 0.40202581826607475,
 'To Heart Omake': 0.4271426754758034,
 'To Heart 2 Adnext': 0.4333862988611539}

In [203]:
# recommendation_for all is used if not enough data
animes_popular = anime_df.sort_values('Completed', ascending=False)[0:top_animes_count]
top_users = rating_popular_anime.user_id.unique()
recommendation_for_all = animes_popular[:100].sort_values(by='Score', ascending=False)[:10]

In [204]:
def recommend_for_user(user_id_input):
    '''
    Take user_id to get top-rated anime by the user, then for each anime make prediction
    return the highest 10 correlation anime
    :param user_id_input: user id
    :return: dataframe of recommended anime
    '''
    recommendation_df = pd.DataFrame(columns=['MAL_ID'])
    users_groups = rating_popular_anime.groupby('user_id')
    if user_id_input in top_users:
        user_fav_anime = users_groups.get_group(user_id_input).sort_values(by='rating', ascending=False)[:5]['MAL_ID'].tolist()
        for user_fav_MAL_ID in user_fav_anime:
            anime_name=anime_df[anime_df['MAL_ID']==user_fav_MAL_ID]['Name'].values[0]
            r = predict(anime_name, 10)
            r_df = anime_df[(anime_df['Name'].isin(r.keys()))]
            distances = []
            for index, row in r_df.iterrows():
                distances.append(r[row['Name']])
            r_df.insert(2, "Distance", distances, True)
            recommendation_df = recommendation_df.append(r_df, ignore_index=True)

        recommendation_df = recommendation_df.sort_values(by='Distance', ascending=True).drop_duplicates('MAL_ID')
        return recommendation_df[:10]
    else:
        return recommendation_for_all

In [205]:
recommend_for_user(2162)

,MAL_ID,Name,Distance,Score,Genres,Type,Episodes,Premiered,Studios,Source,Rating,Members,Completed
23,2119,Captain Tsubasa: Ayaushi! Zen Nihon Jr.,0.069431,6.75,"Action, Sports, Shounen",Movie,1,Unknown,Unknown,Unknown,G - All Ages,4067.0,3055.0
32,2118,Captain Tsubasa: Europe Daikessen,0.069431,6.9,"Action, Sports, Shounen",Movie,1,Unknown,Unknown,Unknown,G - All Ages,4868.0,3801.0
34,2121,Captain Tsubasa: Sekai Daikessen!! Jr. World Cup,0.088282,6.82,"Action, Shounen, Sports",Movie,1,Unknown,Unknown,Unknown,G - All Ages,4053.0,3040.0
33,2120,Captain Tsubasa: Asu ni Mukatte Hashire!,0.098257,6.7,"Action, Sports, Shounen",Movie,1,Unknown,Unknown,Unknown,G - All Ages,3960.0,2970.0
26,2122,Captain Tsubasa: Saikyou no Teki! Holland Youth,0.171598,6.86,"Action, Shounen, Sports",OVA,1,Unknown,Unknown,Unknown,G - All Ages,3850.0,2861.0
11,1674,Captain Tsubasa J,0.281527,7.27,"Action, Sports, Shounen",TV,47,Fall 1994,Studio Comet,Manga,G - All Ages,23423.0,20062.0
10,1614,Captain Tsubasa: Road to 2002,0.336235,7.4,"Action, Sports, Shounen",TV,52,Fall 2001,"Madhouse, Group TAC",Manga,PG-13 - Teens 13 or older,41313.0,35329.0
12,2116,Captain Tsubasa,0.348625,7.3,"Action, Shounen, Sports",TV,128,Fall 1983,Tsuchida Productions,Manga,G - All Ages,65879.0,52195.0
31,2117,Shin Captain Tsubasa,0.373661,7.14,"Action, Sports, Shounen",OVA,13,Unknown,Nippon Animation,Manga,G - All Ages,9537.0,7754.0
2,154,Shaman King,0.590203,7.78,"Action, Adventure, Comedy, Super Power, Supernatural, Shounen",TV,64,Summer 2001,Xebec,Manga,PG-13 - Teens 13 or older,294021.0,208822.0
